In [1]:
# boilerplate
import numpy as np
import tensorflow as tf

sess = tf.InteractiveSession()
import tensorflow_fold as td

# Basic Blocks

In [2]:
scalar = td.Scalar()

In [3]:
scalar.eval(2)

array(2.0, dtype=float32)

In [4]:
vector = td.Vector(3)

In [5]:
vector.eval(np.array([1,2,3]))

array([ 1.,  2.,  3.], dtype=float32)

In [6]:
scal_vec = td.Record((scalar, vector))

In [7]:
scal_vec.eval((4, [1,2,0]))

(array(4.0, dtype=float32), array([ 1.,  2.,  0.], dtype=float32))

# Function Composition

In [12]:
add_1 = td.Scalar() >> td.Function(lambda x: x+1)

In [13]:
# td.Function?

In [14]:
add_1.eval(1)

array(2.0, dtype=float32)

# Sequences

In [15]:
add_1_to_sequence = td.Map(
    td.Scalar() >> td.Function(lambda x: x+1)
)

In [16]:
add_1_to_sequence

<td.Map element_block=<td.Pipe>>

In [17]:
add_1_to_sequence.eval([1,2,3,4,5])

[array(2.0, dtype=float32),
 array(3.0, dtype=float32),
 array(4.0, dtype=float32),
 array(5.0, dtype=float32),
 array(6.0, dtype=float32)]

In [18]:
VEC1 = 0
VEC3 = 1
in_data = [{'type': VEC1, 'data': [1]}, {'type': VEC3, 'data': [0,2,1]}]

In [19]:
features = td.OneOf(
    lambda x: x['type'],
    {
        VEC1: td.GetItem('data') >> td.Vector(1) >> td.FC(5),
        VEC3: td.GetItem('data') >> td.Vector(3) >> td.FC(5)
    }
)

In [20]:
feature_seq = td.Map(features)

In [21]:
feature_seq.eval(in_data)

[array([ 1.39804053,  0.        ,  0.        ,  0.        ,  0.        ], dtype=float32),
 array([ 0.        ,  0.5776673 ,  0.        ,  0.3450402 ,  0.10488975], dtype=float32)]

# Everything becomes part of computational graph

In [22]:
ff_fc_nnet = td.Vector(3) >> td.FC(10) >> td.FC(1, activation=None)

In [23]:
ff_fc_nnet.eval([1,2,3])

array([ 0.85362536], dtype=float32)

In [24]:
def get_input_target_sample():
    X = np.random.randn(3)
    y = X.sum(keepdims=True)
    return X, y

In [25]:
get_input_target_sample()

(array([ 0.85522159,  0.22325032,  0.39455206]), array([ 1.47302397]))

In [26]:
target = td.Vector(1)

In [27]:
compiler = td.Compiler.create((ff_fc_nnet, target))

In [28]:
[y_hat, y] = compiler.output_tensors
loss = tf.reduce_mean(tf.abs(y - y_hat))
train_op = tf.train.AdamOptimizer().minimize(loss)
sess.run(tf.global_variables_initializer())

/home/andershuss/.virtualenvs/tensorflow-fold/local/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [29]:
n_validation_samples = 100
validation_fd = compiler.build_feed_dict(
    [get_input_target_sample() for i in range(n_validation_samples)]
)

In [30]:
def train(n_batches, batch_size, validate_every=10):
    for i in range(n_batches):
        sess.run(
            train_op, compiler.build_feed_dict(
                [get_input_target_sample() for _ in range(batch_size)]
            )
        )
        if (i % validate_every) == 0:
            print(i, sess.run(loss, validation_fd))

In [31]:
train(n_batches=1000, batch_size=64, validate_every=100)

(0, 1.9203987)
(100, 1.4191959)
(200, 0.93930328)
(300, 0.5260759)
(400, 0.38485232)
(500, 0.301036)
(600, 0.24519442)
(700, 0.19815467)
(800, 0.16037312)
(900, 0.13392553)
